In [ ]:
#pbar = ProgressBar(); pbar.register()
import pandas as pd
import numpy as np


In [ ]:
#dic = {
#    'Provider': 'category',
#    'Race_mode': 'category',
#    'State_mode': 'category',
#    'County_mode': 'category',
#    'PotentialFraud': 'category',
#    'Bene_Age_Sum': 'float64',
#    'TotalClaims': 'float64',
#    'AttendingPhysician_TotalClaims': 'float64',
#    'Prv_Physician_Count': 'float64',
#    'Provider_Insurance_Claim_Reimbursement_Amt': 'float64',
#    'Provider_Total_Patients': 'float64',
#    'Provider_Total_ChronicCond_Alzheimer_Patients': 'float64',
#    'Provider_Total_ChronicCond_Heartfailure_Patients': 'float64',
#    'Provider_Total_ChronicCond_KidneyDisease_Patients': 'float64',
#    'Provider_Total_ChronicCond_Cancer_Patients': 'float64',
#    'Provider_Total_ChronicCond_ObstrPulmonary_Patients': 'float64',
#    'Provider_Total_ChronicCond_Depression_Patients': 'float64',
#    'Provider_Total_ChronicCond_Diabetes_Patients': 'float64',
#    'Provider_Total_ChronicCond_IschemicHeart_Patients': 'float64',
#    'Provider_Total_ChronicCond_Osteoporasis_Patients': 'float64',
#    'Provider_Total_ChronicCond_rheumatoidarthritis_Patients': 'float64',
#    'Provider_Total_ChronicCond_stroke_Patients': 'float64',
#    'ClmAdmitDiagnosisCode_Count': 'float64',
#    'ClmDiagnosisCode_1_Count': 'float64',
#    'ClmDiagnosisCode_2_Count': 'float64',
#    'ClmDiagnosisCode_3_Count': 'float64',
#    'ClmAdmitDiagnosisCode_Most_Frequent': 'category',
#    'ClmDiagnosisCode_1_Most_Frequent': 'category',
#    'ClmDiagnosisCode_2_Most_Frequent': 'category',
#    'ClmDiagnosisCode_3_Most_Frequent': 'category',
#    'AttendingPhysician_Most_Frequent': 'category',
#    'OperatingPhysician_Most_Frequent': 'category',
#    'OtherPhysician_Most_Frequent': 'category',
#    'Avg_allocated_Amount_Per_Provider': 'float64',
#    'Avg_Deductible_Amt_Paid_Per_Provider': 'float64',
#    'Avg_InscClaimAmtReimbursed_Per_Provider': 'float64',
#    'perc_allocated_used': 'float64',
#    'prv_avg_claims': 'float64',
#    'prv_avg_claim_cost_indicator': 'float64',
#    'prv_avg_claims_indicator': 'float64'
#}
#

In [1]:
import pandas as pd
import fsspec

# Path to S3 bucket with wildcard
s3_path = "s3://medicare-fraud-data-25-05-2025/merged_ready/train/*.csv"

# Open all matching files with expand=True
fs = fsspec.filesystem("s3")
files = fs.glob("medicare-fraud-data-25-05-2025/merged_ready/train/*.csv")

# Prepend 's3://' to each file path
file_paths = [f"s3://{file}" for file in files]

# Load and concatenate all CSVs into a single DataFrame
df_train = pd.concat([pd.read_csv(f, storage_options={"anon": False}) for f in file_paths], ignore_index=True)


In [2]:
# Path to S3 bucket with wildcard
s3_path = "s3://medicare-fraud-data-25-05-2025/merged_ready/test/*.csv"

# Open all matching files with expand=True
fs = fsspec.filesystem("s3")
files = fs.glob("medicare-fraud-data-25-05-2025/merged_ready/test/*.csv")

# Prepend 's3://' to each file path
file_paths = [f"s3://{file}" for file in files]

# Load and concatenate all CSVs into a single DataFrame
df_test = pd.concat([pd.read_csv(f, storage_options={"anon": False}) for f in file_paths], ignore_index=True)

In [ ]:
#df_testN= df_test.drop(columns=['avg_cost_per_claim'], errors='ignore')

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

class MedicarePreprocessorPandas:
    def __init__(self):
        self.cat_cols = []
        self.num_cols = []
        self.target_col = "PotentialFraud"
        self.fill_values = {}
        self.scaler = None
        self.fitted_columns = []

    def fit(self, df: pd.DataFrame):
        self.cat_cols = [
            "Race_mode", "State_mode", "County_mode",
            "ClmAdmitDiagnosisCode_Most_Frequent",
            "ClmDiagnosisCode_1_Most_Frequent",
            "ClmDiagnosisCode_2_Most_Frequent",
            "ClmDiagnosisCode_3_Most_Frequent",
            "AttendingPhysician_Most_Frequent",
            "OperatingPhysician_Most_Frequent",
            "OtherPhysician_Most_Frequent"
        ]

        df = df.copy()

        # Create engineered features early
        df["avg_cost_per_claim"] = np.where(
            df["TotalClaims"] == 0,
            0.0,
            df["Provider_Insurance_Claim_Reimbursement_Amt"] / df["TotalClaims"]
        )
        df["perc_chronic_alz"] = np.where(
            df["Provider_Total_Patients"] == 0,
            0.0,
            df["Provider_Total_ChronicCond_Alzheimer_Patients"] / df["Provider_Total_Patients"]
        )

        self.num_cols = [
            col for col in df.select_dtypes(include=[np.number]).columns
            if col not in self.cat_cols and col != self.target_col
        ]

        # Save fill values
        self.fill_values = {col: df[col].mean() for col in self.num_cols}
        for col in self.cat_cols:
            self.fill_values[col] = "MISSING"

        # Fill missing and encode
        for col in self.cat_cols:
            df[col] = df[col].astype("category")
            if "MISSING" not in df[col].cat.categories:
                df[col] = df[col].cat.add_categories(["MISSING"])
            df[col] = df[col].fillna("MISSING")
            df[col] = df[col].cat.codes

        for col in self.num_cols:
            df[col] = df[col].fillna(self.fill_values[col])

        self.scaler = StandardScaler()
        self.scaler.fit(df[self.num_cols])

        self.fitted_columns = self.cat_cols + self.num_cols + ["Provider"]
        return self

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()

        # Add missing columns from training
        for col in self.cat_cols + self.num_cols:
            if col not in df.columns:
                if col in self.cat_cols:
                    df[col] = "MISSING"
                else:
                    df[col] = self.fill_values.get(col, 0.0)

        # Feature engineering
        df["avg_cost_per_claim"] = np.where(
            df["TotalClaims"] == 0,
            0.0,
            df["Provider_Insurance_Claim_Reimbursement_Amt"] / df["TotalClaims"]
        )

        df["perc_chronic_alz"] = np.where(
            df["Provider_Total_Patients"] == 0,
            0.0,
            df["Provider_Total_ChronicCond_Alzheimer_Patients"] / df["Provider_Total_Patients"]
        )

        # Fill and encode categoricals
        for col in self.cat_cols:
            df[col] = df[col].astype("category")
            if "MISSING" not in df[col].cat.categories:
                df[col] = df[col].cat.add_categories(["MISSING"])
            df[col] = df[col].fillna("MISSING")
            df[col] = df[col].cat.codes

        for col in self.num_cols:
            df[col] = df[col].fillna(self.fill_values.get(col, 0.0))

        # Scale
        for col in self.num_cols:
            if col not in df.columns:
                df[col] = 0.0
        df[self.num_cols] = self.scaler.transform(df[self.num_cols])

        # Final columns
        final_cols = [col for col in self.fitted_columns if col in df.columns]
        df_out = df[final_cols].copy()

        if self.target_col in df.columns:
            df_out[self.target_col] = df[self.target_col]

        return df_out


In [4]:
# Fit on training data
# Assuming df_train and df_test are pandas DataFrames

preprocessor = MedicarePreprocessorPandas()
preprocessor.fit(df_train)

df_train_processed = preprocessor.transform(df_train)
df_test_processed = preprocessor.transform(df_test)



In [6]:
df_train_processed

,Race_mode,State_mode,County_mode,ClmAdmitDiagnosisCode_Most_Frequent,ClmDiagnosisCode_1_Most_Frequent,ClmDiagnosisCode_2_Most_Frequent,ClmDiagnosisCode_3_Most_Frequent,AttendingPhysician_Most_Frequent,OperatingPhysician_Most_Frequent,OtherPhysician_Most_Frequent,...,Avg_Deductible_Amt_Paid_Per_Provider,Avg_InscClaimAmtReimbursed_Per_Provider,perc_allocated_used,prv_avg_claims,prv_avg_claim_cost_indicator,prv_avg_claims_indicator,avg_cost_per_claim,perc_chronic_alz,Provider,PotentialFraud
0,0,9,19,191,277,264,694,2657,0,0,...,-0.375682,-0.423120,0.132287,-0.539095,-0.156164,-0.539095,-0.428244,-0.487604,PRV52145,0
1,0,32,3,768,277,264,142,289,0,0,...,0.065578,-0.133897,-0.713648,-0.366773,-0.327462,-0.366773,-0.417469,0.229200,PRV55104,0
2,0,32,64,472,11,264,271,4780,0,0,...,1.397284,1.513495,0.589595,-0.431634,6.155988,-0.431634,1.702587,0.036966,PRV54894,1
3,0,32,3,740,276,264,271,1471,0,0,...,-0.226785,0.031728,-0.199320,-0.451401,-0.298733,-0.451401,-0.436006,-0.198213,PRV54927,0
4,0,33,2,768,276,264,271,1127,0,0,...,-0.255899,-0.235811,-0.235060,-0.075958,8.119058,-0.075958,-0.306348,-0.075852,PRV55215,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,0,38,100,780,1018,629,723,976,0,0,...,-1.186361,-1.150955,1.007010,-0.610294,-0.382689,-0.610294,-0.488119,-1.621051,PRV56056,0
5406,0,30,60,282,442,513,177,4117,0,0,...,1.129992,0.861500,-2.123356,-0.610294,-0.338180,-0.610294,2.944568,-1.621051,PRV54820,0
5407,1,38,100,780,390,629,723,344,0,23,...,-1.186361,-1.009326,1.007010,-0.610294,-0.382466,-0.610294,-0.470898,-1.621051,PRV56029,0
5408,3,4,66,780,1168,386,723,861,0,48,...,-1.186361,-0.684072,0.459089,-0.610294,-0.379488,-0.610294,-0.370443,-1.621051,PRV51751,0


In [5]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score
import pandas as pd

# Define features/target
X = df_train_processed.drop(columns=["PotentialFraud", "Provider"])
y = df_train_processed["PotentialFraud"].astype(int)  # ensure it's int if bool/object

# Define scoring
scorer = make_scorer(recall_score , average='binary')

# Cross-validation split strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Model
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    verbosity=0
)

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Grid Search
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    verbose=2,
    n_jobs=-1
)

# Fit
grid_search.fit(X, y)

# Best results
print("Best Parameters:", grid_search.best_params_)
print("Best Recall_Score:", grid_search.best_score_)

# Best model
best_model = grid_search.best_estimator_


NameError: name 'df_train_processed' is not defined

In [19]:
import numpy as np

parameters = {
    'max_depth': 4,
    'learning_rate': 0.1,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'scale_pos_weight': 8.5
}


In [7]:
from sklearn.model_selection import train_test_split

X = df_train_processed.drop(columns=["PotentialFraud"])
y = df_train_processed["PotentialFraud"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


In [8]:
print("Columns in X_train:", X_train.columns.tolist())


Columns in X_train: ['Race_mode', 'State_mode', 'County_mode', 'ClmAdmitDiagnosisCode_Most_Frequent', 'ClmDiagnosisCode_1_Most_Frequent', 'ClmDiagnosisCode_2_Most_Frequent', 'ClmDiagnosisCode_3_Most_Frequent', 'AttendingPhysician_Most_Frequent', 'OperatingPhysician_Most_Frequent', 'OtherPhysician_Most_Frequent', 'ClaimDuration_sum', 'ClaimDuration_mean', 'ClaimDuration_std', 'ClaimDuration_max', 'ClaimDuration_min', 'HospitalDuration_sum', 'HospitalDuration_mean', 'HospitalDuration_std', 'HospitalDuration_max', 'HospitalDuration_min', 'DeductibleAmtPaid_sum', 'DeductibleAmtPaid_mean', 'DeductibleAmtPaid_std', 'DeductibleAmtPaid_max', 'DeductibleAmtPaid_min', 'IPAnnualReimbursementAmt_mean', 'IPAnnualReimbursementAmt_max', 'IPAnnualDeductibleAmt_mean', 'IPAnnualDeductibleAmt_max', 'OPAnnualReimbursementAmt_mean', 'OPAnnualReimbursementAmt_max', 'OPAnnualDeductibleAmt_mean', 'OPAnnualDeductibleAmt_max', 'ChronicCond_Alzheimer_sum', 'ChronicCond_Alzheimer_mean', 'ChronicCond_Heartfailure_

In [9]:
from xgboost import XGBClassifier

X_train['Provider'] = X_train['Provider'].astype('category')
X_test['Provider'] = X_test['Provider'].astype('category')

model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    enable_categorical=True  # This is needed for native categorical support
)

model.fit(X_train, y_train)



/home/ec2-user/.venv/lib64/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:13:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [10]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score
import pandas as pd


In [11]:

from sklearn.metrics import recall_score

# Get predicted probabilities
y_prob = model.predict_proba(X_test)[:, 1]

# Set your custom threshold
threshold = 0.05  # You can adjust this for better recall

# Convert probabilities to class labels
y_pred_thresh = (y_prob >= threshold).astype(int)

# Evaluate
recall = recall_score(y_test, y_pred_thresh)
print(f"Recall (threshold={threshold}):", recall)




Recall (threshold=0.05): 0.8811881188118812


In [12]:
recall = recall_score(y_test, y_pred_thresh)
precision = precision_score(y_test, y_pred_thresh)
f1 = f1_score(y_test, y_pred_thresh)
accuracy = accuracy_score(y_test, y_pred_thresh)
roc_auc = roc_auc_score(y_test, y_prob)


In [13]:
print("Recall Score:", recall)
print("Precision Score:", precision)
print("F1 Score:", f1)
print("Accuracy Score:", accuracy)
print("ROC AUC Score:", roc_auc)
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("\nClassification Report:\n", classification_report(y_test, y_pred_thresh))


Recall Score: 0.8811881188118812
Precision Score: 0.4810810810810811
F1 Score: 0.6223776223776224
Accuracy Score: 0.9001848428835489
ROC AUC Score: 0.9560763415791119

Confusion Matrix:
 [[885  96]
 [ 12  89]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.90      0.94       981
           1       0.48      0.88      0.62       101

    accuracy                           0.90      1082
   macro avg       0.73      0.89      0.78      1082
weighted avg       0.94      0.90      0.91      1082



In [14]:
X = df_train.drop("PotentialFraud", axis=1)
y = df_train["PotentialFraud"]


In [15]:
X['Provider'] = X['Provider'].astype('category')  # Or any other categorical feature


In [17]:
# Convert all object-type columns to category
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].astype('category')


In [18]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    enable_categorical=True  # Use only with categorical dtypes
)

model.fit(X, y)


/home/ec2-user/.venv/lib64/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:26:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)